In [ ]:
import pandas as pd
import plotly.express as px
import json

In [ ]:
with open("../../src/data/landkreise_simplify200.geojson", 'r', encoding='utf-8') as f:
    geojson_kreise= json.load(f)
    print("Geojson geladen")

In [ ]:
df=pd.read_csv("../../src/data/df_lade_bev.csv")
df["Kreis_code"] = df["Kreis_code"].astype(str).str.zfill(5)

In [ ]:
df.head()

In [ ]:
df=df.drop_duplicates(subset="Kreis_code",keep="first")

In [ ]:
df.shape

In [ ]:
df["NL_ratio"]=(df["NL"]/df["Anzahl_Ladepunkte_0425"])
df["SL_ratio"]=(df["SL"]/df["Anzahl_Ladepunkte_0425"])

In [ ]:
df["BEV_pro_Ladepunkt"].describe()

In [ ]:
sum_pkw=df["Bestand_PKW"].sum()

In [ ]:
sum_lp=df["Anzahl_Ladepunkte_0425"].sum()

In [ ]:
# pkw / pkw gesamt

In [ ]:
df["pkw_ratio"]=df["Bestand_PKW"]/sum_pkw

In [ ]:
bev_prog=15*1E6
lp_prog=400*1E3

target_ratio=12





In [ ]:
df["BEV_prog"]=df["pkw_ratio"]*bev_prog
df["BEV_prog_lp_ist"]=df["BEV_prog"]/df["Anzahl_Ladepunkte_0425"]

df["lp_ist_ratio"]=df["Anzahl_Ladepunkte_0425"]/sum_lp

df["lp_prog_linear"]=df["lp_ist_ratio"]*lp_prog
df["BEV_prog_lp_prog_linear"]=df["BEV_prog"]/df["lp_prog_linear"]

df["FQ"]=(df["BEV_prog_lp_prog_linear"]-traget_ratio)**2


In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df["BEV_prog"].sum()

In [ ]:
df["lp_prog_linear"].sum()

In [ ]:
from scipy.optimize import minimize
import numpy as np

In [ ]:
lp_verteilung=df["lp_prog_linear"]
bev_array=df["BEV_prog"]
lp_mindest = df["Anzahl_Ladepunkte_0425"].values 

In [ ]:
def fehlerquadrate(lp_verteilung, bev_array, target_ratio):
    verhältnis = bev_array / lp_verteilung
    fehler = (verhältnis - target_ratio) ** 2
    return np.sum(fehler)

In [ ]:
def summe_ladepunkte(lp_verteilung):
    return np.sum(lp_verteilung) - 400_000 

In [ ]:
fehlerquadrate(lp_verteilung, bev_array, target_ratio)

In [ ]:
lp_start = np.full_like(bev_array, 400_000 / len(bev_array))

In [ ]:
bounds = [(min_val, None) for min_val in lp_mindest]

In [ ]:


res = minimize(
    fehlerquadrate,
    lp_start,
    args=(bev_array, target_ratio),
    method="SLSQP",
    bounds=bounds,
    constraints={"type": "eq", "fun": summe_ladepunkte}
)

In [ ]:
optimierte_lp = res.x  # optimierte Ladepunktverteilung
df["optimierte_Ladepunkte"] = optimierte_lp
df["optimierter_BEV_prog/LP_prog"] = df["BEV_prog"] / df["optimierte_Ladepunkte"]

In [ ]:
df["change_lp%"]=((df["optimierte_Ladepunkte"]-df["Anzahl_Ladepunkte_0425"])/df["Anzahl_Ladepunkte_0425"])*100
df["lp_opt/lp_ist"]=df["optimierte_Ladepunkte"]/df["Anzahl_Ladepunkte_0425"]
df["diff_lp_opt_lp_ist"]=df["optimierte_Ladepunkte"]-df["Anzahl_Ladepunkte_0425"]

In [ ]:
df["optimierte_Ladepunkte"].sum()

In [ ]:
df.head()

In [ ]:
min_val = df["BEV_prog_lp_prog_linear"].min()
max_val = df["BEV_prog_lp_prog_linear"].max()

fig = px.choropleth_map(df,
                           geojson=geojson_kreise, 
                           locations="Kreis_code", 
                           featureidkey="properties.RS",  
                           color="BEV_prog_lp_prog_linear",
                          # color_continuous_scale=color_scale,
                           range_color=[min_val, max_val],
                        
                           #color="status", 
                           #color_discrete_map={"hoch": "red", "mittel": "yellow","niedrig":"green"},
                           color_continuous_scale="Reds",  
                           #range_color=[0,10], #
                           map_style="carto-positron",     
                           zoom=5,                         
                           center={"lat": 51.0, "lon": 10.0}, 
                           opacity=0.8,                       
                           hover_name="Landkreis_y") 
                          # hover_data=hover_data)
                     
fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="∅ BEV_prog_lp_prog_linear",
            font=dict(size=14) )))

fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')

In [ ]:
min_val = df["BEV_prog_lp_prog_linear"].min()
max_val = df["BEV_prog_lp_prog_linear"].max()
#cutoff_low = status_groups["low"]
#cutoff_mid = status_groups["high"]

#max_val_safe = max(max_val, cutoff_mid)

#low_rel = (cutoff_low - min_val) / (max_val_safe - min_val)
#mid_rel = (cutoff_mid - min_val) / (max_val_safe - min_val)




fig = px.choropleth_map(df,
                           geojson=geojson_kreise, 
                           locations="Kreis_code", 
                           featureidkey="properties.RS",  
                           color="optimierter_BEV_prog/LP_prog",
                          # color_continuous_scale=color_scale,
                           range_color=[min_val, max_val],
                        
                           #color="status", 
                           #color_discrete_map={"hoch": "red", "mittel": "yellow","niedrig":"green"},
                           color_continuous_scale="Reds",  
                           #range_color=[0,10], #
                           map_style="carto-positron",     
                           zoom=5,                         
                           center={"lat": 51.0, "lon": 10.0}, 
                           opacity=0.8,                       
                           hover_name="Landkreis_y") 
                          # hover_data=hover_data)
                     
fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="∅ BEV_prog_lp_prog_optimiert",
            font=dict(size=14) )))

fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')

In [ ]:
min_val = df["diff_lp_opt_lp_ist"].min()
max_val = df["diff_lp_opt_lp_ist"].max()
#cutoff_low = status_groups["low"]
#cutoff_mid = status_groups["high"]

#max_val_safe = max(max_val, cutoff_mid)

#low_rel = (cutoff_low - min_val) / (max_val_safe - min_val)
#mid_rel = (cutoff_mid - min_val) / (max_val_safe - min_val)




fig = px.choropleth_map(df,
                           geojson=geojson_kreise, 
                           locations="Kreis_code", 
                           featureidkey="properties.RS",  
                           color="diff_lp_opt_lp_ist",
                          # color_continuous_scale=color_scale,
                           range_color=[min_val, max_val],
                        
                           #color="status", 
                           #color_discrete_map={"hoch": "red", "mittel": "yellow","niedrig":"green"},
                           color_continuous_scale="Reds",  
                           #range_color=[0,10], #
                           map_style="carto-positron",     
                           zoom=5,                         
                           center={"lat": 51.0, "lon": 10.0}, 
                           opacity=0.8,                       
                           hover_name="Landkreis_y") 
                          # hover_data=hover_data)
                     
fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="Notwendige Erhöhung Ladepunkte",
            font=dict(size=14) )))

fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')

In [ ]:
#2. Szenario Ladepunktanzahl nicht beschränkt nach oben, Verhältnis festgesetzt auf 12 BEV/Ladepunkt

In [ ]:
lp_start = np.full_like(bev_array, 400_000 / len(bev_array))

In [ ]:
res = minimize(
    fehlerquadrate,
    x0=lp_start,
    args=(bev_array, target_ratio),
    method="SLSQP",
    bounds=bounds
)

In [ ]:
optimierte_lp_ohne_ob = res.x  # optimierte Ladepunktverteilung
df["optimierte_Ladepunkte_ohne_ob"] = optimierte_lp_ohne_ob
df["optimierter_BEV_prog/LP_prog_ohne_ob"] = df["BEV_prog"] / df["optimierte_Ladepunkte_ohne_ob"]

In [ ]:
df["lp_opt_ohne_ob/lp_ist"]=df["optimierte_Ladepunkte_ohne_ob"]/df["Anzahl_Ladepunkte_0425"]
df["diff_lp_opt_ohne_ob_lp_ist"]=df["optimierte_Ladepunkte_ohne_ob"]-df["Anzahl_Ladepunkte_0425"]

In [ ]:
df["optimierte_Ladepunkte_ohne_ob"].sum()/1E6


In [ ]:
min_val = df["diff_lp_opt_ohne_ob_lp_ist"].min()
max_val = df["diff_lp_opt_ohne_ob_lp_ist"].max()
#cutoff_low = status_groups["low"]
#cutoff_mid = status_groups["high"]

#max_val_safe = max(max_val, cutoff_mid)

#low_rel = (cutoff_low - min_val) / (max_val_safe - min_val)
#mid_rel = (cutoff_mid - min_val) / (max_val_safe - min_val)




fig = px.choropleth_map(df,
                           geojson=geojson_kreise, 
                           locations="Kreis_code", 
                           featureidkey="properties.RS",  
                           color="diff_lp_opt_ohne_ob_lp_ist",
                          # color_continuous_scale=color_scale,
                           range_color=[min_val, max_val],
                        
                           #color="status", 
                           #color_discrete_map={"hoch": "red", "mittel": "yellow","niedrig":"green"},
                           color_continuous_scale="Reds",  
                           #range_color=[0,10], #
                           map_style="carto-positron",     
                           zoom=5,                         
                           center={"lat": 51.0, "lon": 10.0}, 
                           opacity=0.8,                       
                           hover_name="Landkreis_y") 
                          # hover_data=hover_data)
                     
fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="Notwendige Erhöhung Ladepunkte",
            font=dict(size=14) )))

fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')

In [ ]:
min_val = df["optimierter_BEV_prog/LP_prog_ohne_ob"].min()
max_val = df["optimierter_BEV_prog/LP_prog_ohne_ob"].max()
#cutoff_low = status_groups["low"]
#cutoff_mid = status_groups["high"]

#max_val_safe = max(max_val, cutoff_mid)

#low_rel = (cutoff_low - min_val) / (max_val_safe - min_val)
#mid_rel = (cutoff_mid - min_val) / (max_val_safe - min_val)




fig = px.choropleth_map(df,
                           geojson=geojson_kreise, 
                           locations="Kreis_code", 
                           featureidkey="properties.RS",  
                           color="optimierter_BEV_prog/LP_prog_ohne_ob",
                          # color_continuous_scale=color_scale,
                           range_color=[min_val, max_val],
                        
                           #color="status", 
                           #color_discrete_map={"hoch": "red", "mittel": "yellow","niedrig":"green"},
                           color_continuous_scale="Reds",  
                           #range_color=[0,10], #
                           map_style="carto-positron",     
                           zoom=5,                         
                           center={"lat": 51.0, "lon": 10.0}, 
                           opacity=0.8,                       
                           hover_name="Landkreis_y") 
                          # hover_data=hover_data)
                     
fig.update_layout(
    coloraxis_colorbar=dict(
        title=dict(
            text="Notwendige Erhöhung Ladepunkte",
            font=dict(size=14) )))

fig.update_traces( marker_line_width=0,marker_line_color='rgba(0,0,0,0)')